In [2]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


gpg: keybox '/tmp/tmp3q_qnouw/pubring.gpg' created
gpg: /tmp/tmp3q_qnouw/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
import os
os.chdir('drive/nnfl/NNFL/')

In [5]:
!ls

Aleksey_Savrasov      Frida_Kahlo	   resnet50.ipynb
artist_siamese.ipynb  Michelangelo	   Salvador_Dali
Claude_Monet	      Pablo_Picasso	   siam_resnet.h5
code		      Paul_Cezanne	   siam_resnet_with_softmax.h5
comparison.ipynb      Pierre_Renoir	   test.h5
data.h5		      preprocessing.ipynb  tranfer_resnet.h5
data_x.h5	      Rembrandt		   transfer_resnet50.ipynb
data_x.pickle	      resnet50+dropout.h5  Van_Gogh
Da_Vinci	      resnet50.h5	   Wassily_Kandisky


In [0]:
import numpy as np
import cv2

In [16]:
path = os.listdir('.')
path.sort(key=str)
x = np.zeros((2997,224,224,3))
y= np.zeros((2997,12,1))
j = 0
k = 0
for fold in path[:-5]:
  #print(fold)
  path1 = os.listdir(fold)
  path1.sort(key=str)
  for i in path1[:-1]:
    img = cv2.imread(fold+'/'+i,1)
    ran = np.random.randint(0,50)
    img = img[-1-ran-224:-1-ran,-1-ran-224:-1-ran]
    img = np.asarray(img)
    if(len(img.shape) != 3):
      continue
    x[j] = img
    label = np.zeros((12,1))
    label[k] = 1
    y[j] = label
    j = j+1
  k = k+1
  print(j)
  #print(k)  
x = np.asarray(x)
y = np.asarray(y)

169
520
659
726
822
1154
1520
1880
2213
2520
2868


IndexError: ignored

In [0]:
import h5py
hf = h5py.File("test.h5",'w')
#pickle.dump(hf,x)
#hf.close()
hf.create_dataset('dataset_1',data=x)
hf.create_dataset('dataset_2',data=y)
hf.close()

In [0]:
y = y.reshape((2997,12))

In [19]:
import keras

Using TensorFlow backend.


In [0]:
model = keras.models.load_model('resnet50.h5')

In [24]:
model.evaluate(x,y,batch_size = 64)

2997/2997 [==============================] - 38s 13ms/step


[5.196686194505459, 0.167500834197333]

In [25]:
model = keras.models.load_model('resnet50+dropout.h5')
model.evaluate(x,y,batch_size = 64)

2997/2997 [==============================] - 37s 12ms/step


[2.4821190585841886, 0.16716716718208324]

In [29]:
model = keras.models.load_model('tranfer_resnet.h5')
model.evaluate(x,y,batch_size = 64)

2997/2997 [==============================] - 38s 13ms/step


[2.4061360905716964, 0.4037370704634014]

In [30]:
  model = keras.models.load_model('siam_resnet_with_softmax.h5')
model.evaluate(x,y,batch_size = 64)

2997/2997 [==============================] - 37s 13ms/step


[2.381256734286701, 0.10243576910243576]